In [ ]:
!nvidia-smi

Fri Mar 26 05:32:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
cd /content/drive/MyDrive/workspace/lotte/

/content/drive/MyDrive/workspace/lotte


In [ ]:
!unzip ./dataset/LPD_competition.zip -d /content

In [ ]:
!pip install efficientnet_pytorch
!pip install pytorch_pretrained_vit

### 라이브러리 import

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
from efficientnet_pytorch import EfficientNet
from pytorch_pretrained_vit import ViT
from tqdm.auto import tqdm
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import models
from glob import glob
import cv2
from PIL import Image
import torch.nn.functional as F

### Config Setting

In [ ]:
#CONFIG
torch.manual_seed(777)
BATCH_SIZE=100
TTA_SIZE=5
#DEVICE
print(f'PyTorch Version : [{torch.__version__}]')
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'Device : [{device}]')

PyTorch Version : [1.8.0+cu101]
Device : [cuda:0]


### Custom Datasets

In [ ]:
class LotteDataset(Dataset):
  def __init__(self, data_root, train_mode, transform=None,basic_transform=None,TTA=0):
    super(LotteDataset, self).__init__()
    self.train_mode=train_mode
    self.transform=transform
    self.basic_transform=basic_transform
    self.TTA=TTA

    if self.train_mode==False:
      self.img_list = glob(os.path.join(data_root, '*.jpg'))
      self.img_list.sort(key=lambda x:int(x.split('/')[3][:-4]))
    else:
      self.img_list = glob(os.path.join(data_root, '*/*.jpg'))
      self.train_y=[]
      for img_path in self.img_list:
        self.train_y.append(int(img_path.split('/')[3]))
        
    self.len = len(self.img_list)

  def __getitem__(self, index):
    img_path = self.img_list[index]
    # Image Loading
    img = Image.open(img_path)
    if self.TTA>0 :
      imgs=[]
      imgs.append(self.basic_transform(img))
      for _ in range(self.TTA-1):
        imgs.append(self.transform(img))
      return imgs
    else :
      return self.basic_transform(img)

  def __len__(self):
    return self.len

In [ ]:
train_transforms=transforms.Compose([
    transforms.RandomChoice([
        transforms.ColorJitter(brightness=(1,1.1)),
        transforms.ColorJitter(contrast=0.1), 
        transforms.ColorJitter(saturation=0.1),
    ]),
    transforms.RandomChoice([
        transforms.RandomAffine(degrees=15, translate=(0.2, 0.2), scale=(0.8, 1.2), shear=10, resample=Image.BILINEAR,fill=255),
        transforms.RandomCrop((224,224)),
    ]),
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

test_transforms=transforms.Compose([transforms.ToTensor(),
                                    transforms.Resize((224,224)),
                                    transforms.Normalize([0.485, 0.456, 0.406],
                                                          [0.229, 0.224, 0.225])])

test_data=LotteDataset('/content/test',train_mode=False,transform=train_transforms,basic_transform=test_transforms,TTA=TTA_SIZE)
test_iter=DataLoader(test_data,batch_size=BATCH_SIZE,shuffle=False,num_workers=2)

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:1315: UserWarning: Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead
  "Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead"


### TTA Test

In [ ]:
def custom_show(imgs):
  for img in imgs:
    print(img.size())
    img=img.numpy()
    plt.imshow(np.transpose(img,(1,2,0)))
    plt.show()

In [ ]:
for imgs in iter(test_iter):
  for img in imgs:
    custom_show(img)
  break

Output hidden; open in https://colab.research.google.com to view.

### K-Fold Ensemble 기반 다양한 아키텍쳐 Model Ensemble

In [ ]:
class KfoldModel(nn.Module):
  def __init__(self,Fold_1,Fold_2,Fold_3,Fold_4,sharp=True):
    super(KfoldModel,self).__init__()
    self.Fold_1=Fold_1
    self.Fold_2=Fold_2
    self.Fold_3=Fold_3
    self.Fold_4=Fold_4
    self.F_softmax=nn.Softmax(dim=1)

    self.sharp=sharp
    self.T=0.5

  def forward(self,x):
    a=self.Fold_1(x.clone().detach())
    b=self.Fold_2(x.clone().detach())
    c=self.Fold_3(x.clone().detach())
    d=self.Fold_4(x.clone().detach())

    a=self.F_softmax(a)
    b=self.F_softmax(b)
    c=self.F_softmax(c)
    d=self.F_softmax(d)
    if self.sharp:
      return (a**self.T+b**self.T+c**self.T+d**self.T)/4.
    else:
      return (a+b+c+d)/4.

### Resnext50_A (Validation Augmentation)

In [ ]:
#/content/drive/MyDrive/workspace/lotte/checkpoint_resnext50/0_Resnext50_epoch_27.tar
path1='./checkpoint_resnext50/0_Resnext50_epoch_27.tar'
path2='./checkpoint_resnext50/1_Resnext50_epoch_29.tar'
path3='./checkpoint_resnext50/2_Resnext50_epoch_27.tar'
path4='./checkpoint_resnext50/3_Resnext50_epoch_24.tar'

Resnext50_Fold_1=models.resnext50_32x4d(pretrained=False)
checkpoint=torch.load(path1)
Resnext50_Fold_1.load_state_dict(checkpoint['model_state_dict'])
Resnext50_Fold_1.eval()

Resnext50_Fold_2=models.resnext50_32x4d(pretrained=False)
checkpoint=torch.load(path2)
Resnext50_Fold_2.load_state_dict(checkpoint['model_state_dict'])
Resnext50_Fold_2.eval()

Resnext50_Fold_3=models.resnext50_32x4d(pretrained=False)
checkpoint=torch.load(path3)
Resnext50_Fold_3.load_state_dict(checkpoint['model_state_dict'])
Resnext50_Fold_3.eval()

Resnext50_Fold_4=models.resnext50_32x4d(pretrained=False)
checkpoint=torch.load(path4)
Resnext50_Fold_4.load_state_dict(checkpoint['model_state_dict'])
Resnext50_Fold_4.eval()

Model_Resnext50=KfoldModel(Resnext50_Fold_1,Resnext50_Fold_2,Resnext50_Fold_3,Resnext50_Fold_4,sharp=True)
Model_Resnext50.eval()

KfoldModel(
  (Fold_1): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Seq

### EfficientNet b04

In [ ]:
#/content/drive/MyDrive/workspace/lotte/checkpoint_efficient04/0_efficient04_epoch_26.tar
path1='./checkpoint_efficient04/0_efficient04_epoch_26.tar'
path2='./checkpoint_efficient04/1_efficient04_epoch_24.tar'
path3='./checkpoint_efficient04/2_efficient04_epoch_29.tar'
path4='./checkpoint_efficient04/3_efficient04_epoch_23.tar'

Efficient04_Fold_1=EfficientNet.from_name('efficientnet-b4')
checkpoint=torch.load(path1)
Efficient04_Fold_1.load_state_dict(checkpoint['model_state_dict'])
Efficient04_Fold_1.eval()

Efficient04_Fold_2=EfficientNet.from_name('efficientnet-b4')
checkpoint=torch.load(path2)
Efficient04_Fold_2.load_state_dict(checkpoint['model_state_dict'])
Efficient04_Fold_2.eval()

Efficient04_Fold_3=EfficientNet.from_name('efficientnet-b4')
checkpoint=torch.load(path3)
Efficient04_Fold_3.load_state_dict(checkpoint['model_state_dict'])
Efficient04_Fold_3.eval()

Efficient04_Fold_4=EfficientNet.from_name('efficientnet-b4')
checkpoint=torch.load(path4)
Efficient04_Fold_4.load_state_dict(checkpoint['model_state_dict'])
Efficient04_Fold_4.eval()

Model_Efficient04=KfoldModel(Efficient04_Fold_1,Efficient04_Fold_2,Efficient04_Fold_3,Efficient04_Fold_4,sharp=True)
Model_Efficient04.eval()

KfoldModel(
  (Fold_1): EfficientNet(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
    )
    (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
          (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        )
        (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          48, 12, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          12, 48, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_

### Resnext101

In [ ]:
#/content/drive/MyDrive/workspace/lotte/checkpoint_resnext101/0_Resnext_epoch_29.tar
path1='./checkpoint_resnext101/0_Resnext_epoch_29.tar'
path2='./checkpoint_resnext101/1_Resnext_epoch_28.tar'
path3='./checkpoint_resnext101/2_Resnext_epoch_29.tar'
path4='./checkpoint_resnext101/3_Resnext_epoch_27.tar'

Resnext101_Fold_1=models.resnext101_32x8d(pretrained=False)
checkpoint=torch.load(path1)
Resnext101_Fold_1.load_state_dict(checkpoint['model_state_dict'])
Resnext101_Fold_1.eval()

Resnext101_Fold_2=models.resnext101_32x8d(pretrained=False)
checkpoint=torch.load(path2)
Resnext101_Fold_2.load_state_dict(checkpoint['model_state_dict'])
Resnext101_Fold_2.eval()

Resnext101_Fold_3=models.resnext101_32x8d(pretrained=False)
checkpoint=torch.load(path3)
Resnext101_Fold_3.load_state_dict(checkpoint['model_state_dict'])
Resnext101_Fold_3.eval()

Resnext101_Fold_4=models.resnext101_32x8d(pretrained=False)
checkpoint=torch.load(path4)
Resnext101_Fold_4.load_state_dict(checkpoint['model_state_dict'])
Resnext101_Fold_4.eval()

Model_Resnext101=KfoldModel(Resnext101_Fold_1,Resnext101_Fold_2,Resnext101_Fold_3,Resnext101_Fold_4,sharp=True)
Model_Resnext101.eval()

KfoldModel(
  (Fold_1): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Seq

### ViT

In [ ]:
#/content/drive/MyDrive/workspace/lotte/checkpoint_vit/ViT2_epoch_0.tar
path='./checkpoint_vit/ViT2_epoch_0.tar'

Model_ViT = ViT('B_16_imagenet1k', pretrained=False,image_size=224)
checkpoint=torch.load(path)
Model_ViT.load_state_dict(checkpoint['model_state_dict'])
Model_ViT.eval()

ViT(
  (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (positional_embedding): PositionalEmbedding1D()
  (transformer): Transformer(
    (blocks): ModuleList(
      (0): Block(
        (attn): MultiHeadedSelfAttention(
          (proj_q): Linear(in_features=768, out_features=768, bias=True)
          (proj_k): Linear(in_features=768, out_features=768, bias=True)
          (proj_v): Linear(in_features=768, out_features=768, bias=True)
          (drop): Dropout(p=0.1, inplace=False)
        )
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (pwff): PositionWiseFeedForward(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
        )
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (drop): Dropout(p=0.1, inplace=False)
      )
      (1): Block(
 

### All Model

In [ ]:
class FinalModel(nn.Module):
  def __init__(self,Model_1,Model_2,Model_3,Model_4,sharp=True):
    super(FinalModel,self).__init__()
    self.Model_1=Model_1
    self.Model_2=Model_2
    self.Model_3=Model_3
    self.Model_4=Model_4
    self.F_softmax=nn.Softmax(dim=1)

    self.sharp=sharp
    self.T=0.5

  def forward(self,x):
    a=self.Model_1(x.clone().detach())
    b=self.Model_2(x.clone().detach())
    c=self.Model_3(x.clone().detach())
    d=self.F_softmax(self.Model_4(x.clone().detach())) # ViT Model

    if self.sharp:
      return (a**self.T+b**self.T+c**self.T+d**self.T)/4.
    else:
      return (a+b+c+d)/4.

In [ ]:
Model=FinalModel(Model_Resnext50,Model_Resnext101,Model_Efficient04,Model_ViT,sharp=True)
Model.eval()

FinalModel(
  (Model_1): KfoldModel(
    (Fold_1): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         

## Inference

In [ ]:
with torch.no_grad():
  TTA_N=5.0
  Model.eval(), Model.to(device)
  pred_label=[]
  print("Ensemble Inference....\n")
  for imgs in tqdm(iter(test_iter)):
    model_pred=torch.zeros(BATCH_SIZE,1000).to(device)
    for img in imgs:
      img=img.to(device)
      model_pred+=Model(img)

    model_pred=model_pred/TTA_N
    _, y_pred=torch.max(model_pred.data,1)
    pred_label.extend(y_pred.tolist())

submission = pd.read_csv('./dataset/sample.csv', encoding = 'utf-8')
submission['prediction'] = pred_label
submission.to_csv('./submission_TTA5_End_to_End_softvoing.csv', index = False)